# Setup

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
import json
import getpass
import os
import pandas as pd
from docx import Document as DocxDocument
from IPython.display import Image, display
from pydantic import Field
from pydantic import BaseModel
from typing import TypedDict, List, Annotated, Optional, Literal
from langchain import hub
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.docstore.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage,HumanMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import AnyMessage, AIMessage
from fuzzywuzzy import fuzz
from langgraph.checkpoint.memory import MemorySaver
import uuid
import gradio as gr
from langchain_milvus import Milvus
from langfuse.callback import CallbackHandler
import requests
from langchain.chat_models import init_chat_model
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [9]:
langfuse_handler = CallbackHandler(
  secret_key="sk-lf-aac7fcae-34e3-46d6-afc5-247c8ea59682",
  public_key="pk-lf-36ecf322-0d82-4071-acb2-5a77ed6858a5",
  host="https://cloud.langfuse.com"
)

In [10]:
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

llm = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.5)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [11]:
# if not os.environ.get("GOOGLE_API_KEY"):
#   os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

# llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [12]:
URI = "./milvus_example.db"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
    index_params={"index_type": "FLAT", "metric_type": "L2"},
    auto_id=True,
)

In [13]:
# vector_store = InMemoryVectorStore(embeddings)

# Document loader

In [14]:
df_cities = pd.read_csv("/home/jovyan/projects/marin/delta-chatbot/notebooks/Files/cities_per_en.csv")
df_regions = pd.read_csv("/home/jovyan/projects/marin/delta-chatbot/notebooks/Files/regions.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/projects/marin/delta-chatbot/notebooks/Files/cities_per_en.csv'

In [15]:
docx = DocxDocument("/home/jovyan/projects/marin/delta-chatbot/notebooks/Files/delta.docx")
text = "\n".join([para.text for para in docx.paragraphs if para.text.strip()])

docs = [Document(page_content=text, metadata={"source": "/home/jovyan/projects/marin/delta-chatbot/notebooks/Files/delta.docx"})]

PackageNotFoundError: Package not found at '/home/jovyan/projects/marin/delta-chatbot/notebooks/Files/delta.docx'

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=300)
all_splits = text_splitter.split_documents(docs)

In [11]:
_ = vector_store.add_documents(documents=all_splits)

prompt = hub.pull("rlm/rag-prompt")

/opt/conda/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


# Functions & Classes

In [16]:
PROPERTY_TYPE_MAPPING = {
    "زمین": {"url": "store", "display": "زمین"},
    "آپارتمان موقعیت اداری": {"url": "OfficeLocationApartment", "display": "آپارتمان موقعیت اداری"},
    "باغ": {"url": "garden", "display": "باغ"},
    "باغچه": {"url": "garden", "display": "باغ"},
    "تجاری": {"url": "commercial", "display": "تجاری"},
    "مغازه": {"url": "commercial", "display": "مغازه"},
    "آپارتمان": {"url": "apartment", "display": "آپارتمان"}
}
property_types = list(PROPERTY_TYPE_MAPPING.keys())

In [17]:
def fuzzy_search_return_link(neighborhood, threshold=80):
    for idx, row in df_regions.iterrows():
        if neighborhood == row['name']:
            return row['link']
    
    for idx, row in df_regions.iterrows():
        similarity = fuzz.partial_ratio(neighborhood, row['name'])
        if similarity >= threshold:
            return row['link']
    return None

In [18]:
def get_delta_values(value, category, property_type):
    response = requests.get(f"https://api.deltadev.ir/api/v1/customFields/filter?category={category}&property={property_type}")
    response.raise_for_status()
    data = response.json()
    for item in data["data"]:
        if item["key"] == value:
            items = [row["value"] for row in item["options"]]
    items = [int(x) for x in items]
    return items

def value_assignment(min_value, max_value, a):
    if max_value:
        value = max_value
    else:
        value = min_value
        
    max_value = None
    min_value = None
    
    value = min(a, key=lambda x: abs(x - value))
    index = a.index(value)
    
    if index != len(a)-1 and index !=0:
        max_value = a[index+1]
        min_value = a[index-1]
        
    elif index == len(a)-1:
        min_value = value
        max_value = None
        
    elif index == 0:
        if a[0]==0:
            max_value = a[1]
        else:
            max_value = a[0]
        min_value = None
    return min_value, max_value


def set_values(value, min_value, max_value, is_value_approximate, category, property_type):
    if is_value_approximate==True or min_value == max_value:
        values = get_delta_values(value, category, property_type)
        if (min_value and not max_value) or (not min_value and max_value):
            min_value, max_value = value_assignment(min_value, max_value, values)
        elif min_value==None and max_value==None:
            pass
        else:
            min_value = min(values, key=lambda x: abs(x - min_value))
            max_value = min(values, key=lambda x: abs(x - max_value))
            if min_value == max_value:
                min_value, max_value = value_assignment(min_value, None, values)
    return min_value, max_value
        
def search_properties(
    min_meter: Optional[int] = None, 
    max_meter: Optional[int] = None,
    is_meter_approximate: Optional[bool] = None,
    min_price: Optional[int] = None, 
    max_price: Optional[int] = None,
    is_price_approximate: Optional[bool] = None,
    min_rent_price: Optional[int] = None, 
    max_rent_price: Optional[int] = None, 
    is_rent_price_approximate: Optional[bool] = None,
    neighborhood: Optional[str] = None,
    city: Optional[str] = "tehran",
    elevator: Optional[bool] = None,
    property_type: Optional[str] = None,
    category: Optional[str] = None,
) -> str:
    """
    if the user wants to buy or rent property make url with filters for city, area, price, elevator, and property type.
    """
    print(min_meter)
    print(max_meter)
    if not property_type:
        property_type = apartment
    else:
        property_type = PROPERTY_TYPE_MAPPING[property_type]["url"]

    if not category:
        category = "buy"
        
    base_url = f"https://delta.ir/{city}/{category}/{property_type}"

    if neighborhood:
        fuzzy_result = fuzzy_search_return_link(neighborhood)
        if fuzzy_result:
            base_url = base_url + '/' + fuzzy_result

    
    min_meter, max_meter = set_values("Area", min_meter, max_meter, is_meter_approximate, category, property_type)
    min_price, max_price = set_values("Price", min_price, max_price, is_price_approximate, category, property_type)
    if category == "rent":
        min_rent_price, max_rent_price = set_values("RentalPrice", min_rent_price, max_rent_price, is_rent_price_approximate, category, property_type)      

            
    query_params = []
    if min_meter is not None and max_meter is not None:
        query_params.append(f"meter={min_meter}-{max_meter}")
    elif min_meter is not None:
        query_params.append(f"meter={min_meter}-")
    elif max_meter is not None:
        query_params.append(f"meter=-{max_meter}")

    if min_price is not None and max_price is not None:
        query_params.append(f"price={min_price}-{max_price}")
    elif min_price is not None:
        query_params.append(f"price={min_price}-")
    elif max_price is not None:
        query_params.append(f"price=-{max_price}")

    if elevator==True:
        query_params.append("features=elevator")

    query_string = "?" + "&".join(query_params) if query_params else ""
    url = base_url + query_string

    filters = []
    if min_meter is not None and max_meter is not None:
        filters.append(f"{min_meter} تا {max_meter} متر")
    elif min_meter is not None:
        filters.append(f"حداقل {min_meter} متر")
    elif max_meter is not None:
        filters.append(f"حداکثر {max_meter} متر")

    if min_price is not None and max_price is not None:
        filters.append(f"بین {min_price:,} تا {max_price:,} تومان")
    elif min_price is not None:
        filters.append(f"حداقل {min_price:,} تومان")
    elif max_price is not None:
        filters.append(f"حداکثر {max_price:,} تومان")

    if elevator:
        filters.append("با آسانسور")

    if filters:
        filters_str = "، ".join(filters)
        explanation = f"با ویژگی‌هایی که گفتید ({filters_str})، می‌توانید بر روی این لینک کلیک کنید و ملک‌های مشابه را پیدا کنید: {url}"
    else:
        explanation = f"می‌توانید بر روی این لینک کلیک کنید و ملک‌های موجود را مشاهده کنید: {url}"
    return url

In [19]:
from pydantic import BaseModel, Field, field_validator
from typing import Optional

class PropertyFeatures(BaseModel):
    min_meter: Optional[int] = Field(default=None, description="Minimum square meters")
    max_meter: Optional[int] = Field(default=None, description="Maximum square meters")
    is_meter_approximate: Optional[bool] = Field(
        default=None,
        description="Indicates whether the meterage is approximate (e.g., 'around 100 meters') and set to True, or an open range (e.g., '100 meters and above' or 'from 100 meters') and set to False."
    )
    
    min_price: Optional[int] = Field(default=None, description="Minimum price or Security Deposit if category is rent in Tomans")
    max_price: Optional[int] = Field(default=None, description="Maximum price or Security Deposit if category is rent in Tomans")
    is_price_approximate: Optional[bool] = Field(
        default=None,
        description="Indicates whether the price (or Security Deposit ( or 'ودیعه' in persian) if category is rent) is approximate (e.g., 'around 2 billion' or 'something like 2 billion') and set to True, or an open range (e.g., 'from 2 billion' or '2 billion and above') and set to False."
    )
    
    min_rent_price: Optional[int] = Field(default=None, description="Minimum monthly rental price in Tomans")
    max_rent_price: Optional[int] = Field(default=None, description="Maximum monthly rental price in Tomans")
    is_rent_price_approximate: Optional[bool] = Field(
        default=None,
        description="Indicates whether the monthly rental price is approximate (e.g., 'around 2 million' or 'something like 2 million') and set to True, or an open range (e.g., 'from 2 million' or '2 million and above') and set to False."
    )
    
    neighborhood: Optional[str] = Field(default=None, description="Neighborhood in tehran in persian")
    city: Literal[tuple(df_cities['city'])] = Field(default="other", description="City name. city='rasht' if city is shomal")
    elevator: Optional[bool] = Field(default=None, description="Elevator required (True/False)")
    property_type: Literal[*property_types]= Field(default=None, description="property type") 
    category: Literal["buy", "rent"] = Field(default=None, description="Transaction type: 'buy' for purchase, 'rent' for rental") 
    
    @field_validator("is_meter_approximate", mode="before")
    @classmethod
    def set_meter_approximate(cls, v, info):
        if v is None and info.data.get("min_meter") is not None:
            return True  
        return v

    @field_validator("is_price_approximate", mode="before")
    @classmethod
    def set_price_approximate(cls, v, info):
        if v is None and info.data.get("min_price") is not None:
            return False  
        return v

NameError: name 'df_cities' is not defined

In [20]:
class FollowUpQuestions(BaseModel):
    follow_up_questions: List[str]

In [21]:
from langchain_core.messages import AnyMessage 
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]
    follow_up_questions: List[str]

# Tools

In [22]:
@tool
def retrieve(query: str):
    """Retrieves relevant documents based on the content of the most recent message."""
    print("retrieve tool used ")

    # using threshold
    retrieved_docs = vector_store.similarity_search_with_score(query, k=3)
    filtered_docs = [doc for doc, score in retrieved_docs if score >= 0.4]
    serialized = "\n\n".join(
        (f"Source: \n" f"Content: {doc}")
        for doc in filtered_docs
    )

    return serialized


@tool
def make_buy_or_rent_properties_url(query: str):
    """
    Use this tool to generate a URL for purchasing or renting real estate properties.
    """
    
    structured_llm = llm.with_structured_output(PropertyFeatures)
    property_features = structured_llm.invoke(query)
    print("make_buy_or_rent_properties_url tool used with this parameters: ", property_features)

    result = search_properties(**property_features.model_dump())
    return result
    
tools = [retrieve, make_buy_or_rent_properties_url]

# Chains

In [23]:
def query_or_respond(state: State):
    """Generate tool call or respond."""
    llm_with_tools = llm.bind_tools(tools)
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def generate(state: State):
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]
    print("tool_messages = ", tool_messages)

    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an Real estate consultant of delta.ir for question-answering tasks. "
        "Don't suggest any website or link from other websites except from delta.ir"
        "Use the following pieces of retrieved context to answer Persian. "
        "If you got an url, give the url to user and don't change the url. give url only once, don't repeate it again in a () or []."
        "If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the answer concise. "
        "don't mention the source of the document, only answer it"
        "Provide only the answer, no source or metadata."
        "\n\n"
        f"{docs_content}"
    )
    
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]

    prompt = [SystemMessage(content=system_message_content)] + conversation_messages

    response = llm.invoke(prompt)

    follow_up_system_message = SystemMessage(
        content=(
            "Based on the user's query, suggest 3 relevant follow-up questions in Persian. "
            "Return each question on a new line."
        )
    )
    follow_up_prompt = [follow_up_system_message] + prompt + [response]
    follow_up_response = llm.invoke(follow_up_prompt)

    follow_up_questions = [
        q.strip() for q in follow_up_response.content.split("\n") if q.strip()
    ]

    return {
        "messages": [AIMessage(content=response.content)],
        "follow_up_questions": follow_up_questions
    }


graph_builder = StateGraph(State)
graph_builder.add_node(query_or_respond)
tool_node = ToolNode(tools)
graph_builder.add_node(tool_node)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

# Usage

In [24]:
# Updating memory
config = None
def update_memory():
    thread_id = str(uuid.uuid4())
    config={
        "callbacks": [langfuse_handler],
        "configurable": {"thread_id": thread_id},
        "metadata": {"langfuse_session_id": thread_id},
    }
    return config

def handle_update_memory():
    global config
    config = update_memory()
    return f"حافظه با موفقیت آپدیت شد\n {config['configurable']['thread_id']}", []
handle_update_memory()
print(config)

{'callbacks': [<langfuse.callback.langchain.LangchainCallbackHandler object at 0x76b56596dd00>], 'configurable': {'thread_id': 'f7fbe76d-4bb8-4e22-a41f-3d0042f15c06'}, 'metadata': {'langfuse_session_id': 'f7fbe76d-4bb8-4e22-a41f-3d0042f15c06'}}


# Test Cases

In [25]:
def ask_model(input_message):
    print("input= ", input_message, "\n")
    result = graph.invoke(
        {"messages": [HumanMessage(content=input_message)]},
        config=config
    )

    response = result["messages"][-1].content
    follow_ups = result.get("follow_up_questions", [])
    follow_ups_str = "\n".join(f"- {q}" for q in follow_ups)

    chat_history = []
    for i in result["messages"]:
        if isinstance(i, HumanMessage):
            chat_history.append(("شما", i.content))
        if isinstance(i, AIMessage) and i.content != '':
            chat_history.append(("چت بات", i.content))
            

    return response, follow_ups_str, chat_history

In [26]:
def test_retrieve_null(n):
    input_messages = [
        "میخوام خونمو بفروشم از چه راه هایی بهتره؟",
        "بهترین روش برای فروش ملک در تهران چیست؟",
        "چطور میتونم ماشینم رو سریع بفروشم؟",
        "راهنمایی میخواستم برای فروش ویلا در شمال",
        "آیا مشاور املاک برای فروش خانه لازم است؟"
    ]
    return ask_model(input_messages[n])

def test_retrieve(n):
    input_messages = [
        "تعداد آگهی های مجاز در روز چقدر است؟",
        "در روز چه تعداد آگهی در سایت میتوان ثبت کرد؟",
        "حداکثر آگهی مجاز روزانه چندتاست؟",
        "محدودیت تعداد آگهی به عنوان مشاور مستقل چقدر است؟",
        "آیا برای آگهی های روزانه سقفی وجود دارد؟",
        "آیا میتونم لیست تراکنش هایی که انجام دادم رو ببینم؟",
        "ارتقای آگهی چگونه است؟",
        "ویتیرن آگهی چگونه است و هزینه آن چقدر است؟"
    ]
    return ask_model(input_messages[n])

def test_url(n):
    input_messages = [
        "میخوام یه خونه میدون ولیعصر بخرم بالای 3 میلیارد و حدود 200 متر",  #0
        "دنبال آپارتمان در نیاوران زیر 5 میلیارد میگردم",  #1
        "ویلای ساحلی در شمال زیر 10 میلیارد با زمین 500 متر",  #2
        "آپارتمان 2 خوابه در غرب تهران حدود 2 میلیارد",  #3
        "یه آپارتمان 3 خوابه در زعفرانیه بخرم حدود 8 میلیارد",  #4
        "دنبال خونه ویلایی در لواسان با بودجه 15 میلیارد",  #5
        "آپارتمان 100 متری در سعادت آباد زیر 4 میلیارد",  #6
        "یه واحد 2 خوابه در پونک برای اجاره، ماهی 10 میلیون",  #7
        "ویلا در کردان با زمین 1000 متر زیر 12 میلیارد بخرم",  #8
        "آپارتمان 1 خوابه در یوسف آباد اجاره کنم، ودیعه 200 میلیون، ماهی 8 میلیون",  #9
        "خونه 150 متری در جردن بخرم حدود 6 میلیارد",  #10
        "دنبال آپارتمان نوساز در الهیه با بودجه 10 میلیارد میگردم",  #11
        "یه واحد 80 متری در شهرک غرب میخوام اجاره کنم، با ودیعه 300 میلیون",  #12
        "ویلا در محمودآباد با بودجه 7 میلیارد و حداقل 300 متر زمین",  #13
        "آپارتمان 2 خوابه در گیشا زیر 3 میلیارد بخرم",  #14
        "خونه 120 متری در ونک برای اجاره، ماهی 15 میلیون",  #15
        "دنبال آپارتمان 3 خوابه در فرمانیه با بودجه 12 میلیارد میگردم",  #16
        "میخوام یه واحد 70 متری در تهرانپارس اجاره کنم، ودیعه 150 میلیون، ماهی 6 میلیون",  #17
        "ویلا در رامسر با بودجه 9 میلیارد و ویو دریا",  #18
        "آپارتمان 100 متری در صادقیه زیر 2.5 میلیارد",  #19
        "خونه ویلایی در قیطریه با بودجه 20 میلیارد",  #20
        "یه واحد 90 متری در مرزداران اجاره کنم، ودیعه 250 میلیون، ماهی 9 میلیون",  #21
        "آپارتمان 2 خوابه در ولنجک زیر 7 میلیارد بخرم",  #22
        "دنبال ویلا در نوشهر با بودجه 8 میلیارد و زمین 400 متر",  #23
        "آپارتمان 80 متری در جنت آباد برای اجاره، ماهی 7 میلیون",  #24
        "خونه 200 متری در اقدسیه با بودجه 15 میلیارد بخرم",  #25
        "یه واحد 3 خوابه در پاسداران اجاره کنم، ودیعه 500 میلیون، ماهی 20 میلیون",  #26
        "آپارتمان 1 خوابه در شهران زیر 1.5 میلیارد",  #27
        "ویلا در کلاردشت با بودجه 6 میلیارد و حداقل 500 متر زمین",  #28
        "دنبال آپارتمان 100 متری در تجریش برای اجاره، ودیعه 400 میلیون",  #29
    ]
    return ask_model(input_messages[n])

def test_none_tool(n):
    input_messages = [
        "خرید و فروش تو چه منطقه ای بهتره؟",
        "بهترین محله تهران برای سرمایه گذاری کجاست؟",
        "کدام مناطق مشهد برای خرید ملک مناسبند؟",
        "نظرشما درباره خرید ملک در شهرهای جدید چیست؟",
        "آیا الان زمان خوبی برای خرید خانه است؟"
    ]
    return ask_model(input_messages[n])

def test_unrelated(n):
    input_messages = [
        "به عنوان مشاور املاک برام یه کد به پایتون بنویس که سلام دنیا رو چاپ کنه",
        "نحوه پخت قرمه سبزی چگونه است؟",
        "بهترین مارک یخچال کدام است؟",
        "برنامه نویسی پایتون را چگونه شروع کنم؟",
        "تفاوت بین خودروهای بنزینی و الکتریکی چیست؟"
    ]
    return ask_model(input_messages[n])

In [27]:
with gr.Blocks() as demo:
    gr.Markdown("## مشاور املاک هوشمند")

    chat_history = gr.Chatbot(label="هیستوری چت", height=300)
    input_box = gr.Textbox(label="پیامت رو بنویس")
    ask_button = gr.Button("📤")
    
    response_output = gr.Textbox(label="پاسخ مدل", lines=4)
    follow_up_output = gr.Textbox(label="سوالات پیشنهادی", lines=4)

    update_button = gr.Button("⚠️آپدیت حافظه")
    update_status = gr.Textbox(label="وضعیت حافظه")

    ask_button.click(
        fn=ask_model,
        inputs=[input_box],
        outputs=[response_output, follow_up_output, chat_history]
    )

    update_button.click(
        fn=handle_update_memory,
        outputs=[update_status, chat_history]
    )

demo.launch(share=True)

/tmp/ipykernel_59050/1282336978.py:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat_history = gr.Chatbot(label="هیستوری چت", height=300)


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Missing file: /home/jovyan/.cache/huggingface/gradio/frpc/frpc_linux_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.3
3. Move the file to this location: /home/jovyan/.cache/huggingface/gradio/frpc
